# Autonomous trackbar
This is the autonomous trackbar method.

If not evaluating the method, please set `EVALUATION = False`

In [ ]:
EVALUATION = False

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pytesseract
import subprocess

# Path to tesseract executable (in case it isn't in your PATH)
try:
    subprocess.call(["tesseract"])
except FileNotFoundError:
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

"""
Automatically modify the values (lh, ls, lv, uh, us, uv) to OCR the text in the image.
"""


def auto_trackbar_ocr(image):
    # Ask user for background color
    background_color = int(input("Enter the background color (1 for Dark and 0 for Light): "))

    # Convert BGR to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # define range of boundaries color
    bg_image = np.copy(image)
    bg_image = cv2.GaussianBlur(bg_image, (15, 15), 0)
    bg_image = cv2.medianBlur(bg_image, 31)

    h, s, v = cv2.split(bg_image)
    if background_color == 0:
        lower_value = [np.min(h) % 180, np.min(s), np.min(v)]
        upper_value = [180, 255, 255]
    else:
        lower_value = [0, 0, 0]
        upper_value = [np.max(h) % 180, np.max(s), np.max(v)]

    # lower_value = [49, 34, 216] # values per sandisk image 007
    # upper_value = [179, 60, 255] # values per sandisk image 007

    lower_bound = np.array(lower_value, np.uint8)
    upper_bound = np.array(upper_value, np.uint8)

    # Threshold the HSV to get only the written text
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    mask = cv2.bitwise_not(mask)

    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(image, image, mask=mask)
    if not EVALUATION:
        print("Text before filtering: ")
        print("\033[91m {}\033[00m".format(pytesseract.image_to_string(image)))
    # OCR
    res = cv2.GaussianBlur(res, (9, 9), 0)
    text = pytesseract.image_to_string(res)
    if not EVALUATION:
        print("Text after autonomous trackbar: \033[92m {}\033[00m".format(text))

        plt.subplot(121), plt.imshow(mask), plt.title("Mask")
        plt.xticks([]), plt.yticks([])
        plt.subplot(122), plt.imshow(res), plt.title("Result")
        plt.xticks([]), plt.yticks([])
        plt.show()


In [ ]:
def remove_most_used_bins(image, num_bins_to_remove):
    # Convert image to desired color space (e.g., BGR to HSV)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Calculate histogram
    hist = cv2.calcHist([hsv_image], [0, 1], None, [180, 256], [0, 180, 0, 256])
    color = ('b', 'g', 'r')
    for i, col in enumerate(color):
        histr = cv2.calcHist([hsv_image], [i], None, [256], [0, 256])
        plt.plot(histr, color=col)
        plt.xlim([0, 256])
    plt.show()

    # Flatten histogram
    flat_hist = hist.flatten()

    # Sort histogram bins by frequency
    sorted_bins = np.argsort(-flat_hist)

    # Identify most frequently occurring bins
    most_used_bins = sorted_bins[:num_bins_to_remove]

    # Replace pixels corresponding to most used bins with a color of your choice (e.g., white)
    for bin_idx in most_used_bins:
        row = bin_idx // 256
        col = bin_idx % 256
        hsv_image[np.where((hsv_image[:, :, 0] == row) & (hsv_image[:, :, 1] == col))] = [180, 0, 0]  # Red color


    for i, col in enumerate(color):
        histr = cv2.calcHist([hsv_image], [i], None, [256], [0, 256])
        plt.plot(histr, color=col)
        plt.xlim([0, 256])
    plt.show()
    # Convert back to BGR color space
    result_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)
    result_image = cv2.GaussianBlur(result_image, (11, 11), 0)
    text = pytesseract.image_to_string(result_image)
    if not EVALUATION:
        print("Text after autonomous trackbar: \033[92m {}\033[00m".format(text))

        plt.subplot(121), plt.imshow(image), plt.title("image")
        plt.xticks([]), plt.yticks([])
        plt.subplot(122), plt.imshow(result_image), plt.title("Result")
        plt.xticks([]), plt.yticks([])
        plt.show()

In [ ]:
if __name__ == "__main__":
    plt.rcParams['figure.figsize'] = [15, 10]
    img = cv2.imread('../images/001.jpg')
    assert img is not None, "file could not be read, check with os.path.exists()"
    # auto_trackbar_ocr(img)
    remove_most_used_bins(img, 100)

## Evaluation

In [ ]:
import cv2
import numpy as np
import pytesseract
import os
import subprocess
import pandas as pd
import re
from Levenshtein import distance

if not EVALUATION:
    assert False

# Images for automatic filtering OCR are from 001.jpg to 010.jpg
image_names = [str(i).zfill(3) + ".jpg" for i in range(1, 11)]

# Remove 004.jpg: this image is not meant to be preprocessed as it is already black text on a white background
image_names.remove("004.jpg")

# Hardcoded values for each image: dictionary with image_name as key and values and value 0 or 1
background_colors = {
    "001.jpg": 1,
    "002.jpg": 0,
    "003.jpg": 1,
    "004.jpg": 1,
    "005.jpg": 1,
    "006.jpg": 0,
    "007.jpg": 1,
    "008.jpg": 1,
    "009.jpg": 0,
    "010.jpg": 1
}

PARENT_DIR = os.path.dirname(os.path.dirname(os.path.realpath("FILEPATH")))

# Remove the previous results/autonomous_trackbar.tsv if it exists
if os.path.exists(os.path.join(PARENT_DIR, "results", "autonomous_trackbar.tsv")):
    os.remove(os.path.join(PARENT_DIR, "results", "autonomous_trackbar.tsv"))

for image_name in image_names:
    # Path to tesseract executable (in case it isn't in your PATH)
    try:
        subprocess.call(["tesseract"])
    except FileNotFoundError:
        pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    image = cv2.imread(os.path.join(PARENT_DIR, "images", image_name), cv2.IMREAD_COLOR)

    # Check if image is loaded fine
    if image is None:
        print('Error opening image')

    # background_color = int(input("Enter the background color (1 for Dark and 0 for Light): "))
    background_color = background_colors[image_name]

    # Convert BGR to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # define range of boundaries color
    bg_image = np.copy(image)
    bg_image = cv2.GaussianBlur(bg_image, (15, 15), 0)
    bg_image = cv2.medianBlur(bg_image, 31)

    h, s, v = cv2.split(bg_image)
    if background_color == 0:
        lower_value = [np.min(h) % 180, np.min(s), np.min(v)]
        upper_value = [180, 255, 255]
    else:
        lower_value = [0, 0, 0]
        upper_value = [np.max(h) % 180, np.max(s), np.max(v)]

    lower_bound = np.array(lower_value, np.uint8)
    upper_bound = np.array(upper_value, np.uint8)

    # Threshold the HSV to get only the written text
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    mask = cv2.bitwise_not(mask)

    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(image, image, mask=mask)
    res = cv2.GaussianBlur(res, (9, 9), 0)

    method_text = pytesseract.image_to_string(res)
    if method_text == "":
        method_text = " "

    baseline_text = pytesseract.image_to_string(image)
    if baseline_text == "":
        baseline_text = " "

    # Remove special characters from the text (\n, \t, \r, and multiple spaces all become a single space)
    baseline_text = re.sub(r'[\n\t\r]+', ' ', baseline_text)
    baseline_text = re.sub(r' +', ' ', baseline_text)
    method_text = re.sub(r'[\n\t\r]+', ' ', method_text)
    method_text = re.sub(r' +', ' ', method_text)

    # Load the ground truth (results/ground_truth.tsv)
    ground_truth = pd.read_csv(os.path.join(PARENT_DIR, "results", "ground_truth.tsv"), sep="\t")

    # Create results/autonomous_trackbar.tsv if it doesn't exist or if it has a different number of rows than ground_truth.tsv
    if not os.path.exists(os.path.join(PARENT_DIR, "results", "autonomous_trackbar.tsv")) or len(ground_truth) != len(
            pd.read_csv(os.path.join(PARENT_DIR, "results", "autonomous_trackbar.tsv"), sep="\t")):
        autonomous_trackbar = pd.DataFrame(
            columns=["input", "text", "baseline_text", "method_text", "ocr", "baseline_dist", "method_dist"])
        # Add all rows from ground_truth to autonomous_trackbar, with values for "ocr" set to 0
        autonomous_trackbar["input"] = ground_truth["input"]
        autonomous_trackbar["text"] = ground_truth["text"]
        autonomous_trackbar["baseline_text"] = ""
        autonomous_trackbar["method_text"] = ""
        autonomous_trackbar["ocr"] = 0
        # distance is the Levenshtein distance between the ground truth and the OCR result, defaulting to -1
        autonomous_trackbar["baseline_dist"] = -1
        autonomous_trackbar["method_dist"] = -1
        # Save the autonomous_trackbar dataframe
        autonomous_trackbar.to_csv(os.path.join(PARENT_DIR, "results", "autonomous_trackbar.tsv"), sep="\t",
                                   index=False)

    # Load the autonomous_trackbar.tsv
    autonomous_trackbar = pd.read_csv(os.path.join(PARENT_DIR, "results", "autonomous_trackbar.tsv"), sep="\t",
                                      dtype={"method_text": "string", "baseline_text": "string"})

    # Insert/update the autonomous_trackbar dataframe
    autonomous_trackbar.loc[autonomous_trackbar["input"] == image_name, "text"] = \
    ground_truth[ground_truth["input"] == image_name]["text"].values[0]
    autonomous_trackbar.loc[autonomous_trackbar["input"] == image_name, "baseline_text"] = baseline_text
    autonomous_trackbar.loc[autonomous_trackbar["input"] == image_name, "method_text"] = method_text
    autonomous_trackbar.loc[autonomous_trackbar["input"] == image_name, "ocr"] = 1
    autonomous_trackbar.loc[autonomous_trackbar["input"] == image_name, "baseline_dist"] = distance(
        ground_truth[ground_truth["input"] == image_name]["text"].values[0], baseline_text)
    autonomous_trackbar.loc[autonomous_trackbar["input"] == image_name, "method_dist"] = distance(
        ground_truth[ground_truth["input"] == image_name]["text"].values[0], method_text)

    # Save the autonomous_trackbar dataframe
    autonomous_trackbar.to_csv(os.path.join(PARENT_DIR, "results", "autonomous_trackbar.tsv"), sep="\t", index=False)
    # autonomous_trackbar

# Load the autonomous_trackbar.tsv
autonomous_trackbar = pd.read_csv(os.path.join(PARENT_DIR, "results", "autonomous_trackbar.tsv"), sep="\t")
autonomous_trackbar